In [1]:
import merge_pluto_finance_new as mpf

In [2]:
import pandas as pd
import numpy as np

In [3]:
boros = ["manhattan", "brooklyn", "bronx", "queens", "si"]
years = list(range(2003, 2017))

In [4]:
boros = ["".join(boro.lower().split()) for boro in boros]

In [5]:
pluto = mpf.read_in_pluto(boros)

In [6]:
finance = mpf.read_in_finance(boros, years)

Pulling Finance data for 2003_manhattan
Pulling Finance data for 2003_brooklyn
Pulling Finance data for 2003_bronx
Pulling Finance data for 2003_queens
Pulling Finance data for 2003_si
Pulling Finance data for 2004_manhattan
Pulling Finance data for 2004_brooklyn
Pulling Finance data for 2004_bronx
Pulling Finance data for 2004_queens
Pulling Finance data for 2004_si
Pulling Finance data for 2005_manhattan
Pulling Finance data for 2005_brooklyn
Pulling Finance data for 2005_bronx
Pulling Finance data for 2005_queens
Pulling Finance data for 2005_si
Pulling Finance data for 2006_manhattan
Pulling Finance data for 2006_brooklyn
Pulling Finance data for 2006_bronx
Pulling Finance data for 2006_queens
Pulling Finance data for 2006_si
Pulling Finance data for 2007_manhattan
Pulling Finance data for 2007_brooklyn
Pulling Finance data for 2007_bronx
Pulling Finance data for 2007_queens
Pulling Finance data for 2007_si
Pulling Finance data for 2008_manhattan
Pulling Finance data for 2008_brook

In [9]:
dtm = mpf.read_in_dtm(boros)

In [7]:
pluto.shape[0] - pluto.drop_duplicates().shape[0]

0

In [91]:
finance.shape[0] - finance.drop_duplicates().shape[0]

0

In [10]:
dtm.shape[0] - dtm.drop_duplicates().shape[0]

0

In [14]:
finance_condo_updated = mpf.get_finance_condo_lot(pluto = pluto,
        finance = finance, dtm = dtm)

Finance:(267610, 9) DTM:(228319, 6)
Finance intermediate:(122977, 12) PLUTO:(812364, 41)
Finance intermediate (all condos): (5248, 15)


In [12]:
buildings = mpf.merge_pluto_finance(pluto, finance, dtm)

Updating lot numbers for condo units
Finance:(267610, 9) PLUTO:(812364, 41) DTM:(228319, 6)
Finance:(267610, 9) DTM:(228319, 6)
Finance intermediate:(122977, 12) PLUTO:(812364, 41)
Finance intermediate (all condos): (5248, 15)
Finance updated:(272615, 9)
Merging PLUTO with updated Dept. of Finance data


In [13]:
buildings.shape[0] - buildings.drop_duplicates().shape[0]

4906

In [133]:
finance_condo_updated.shape[0] - finance_condo_updated.drop_duplicates().shape[0]

0

In [114]:
dtm_cols_to_keep = ['unit_bbl', 'condo_boro', 'condo_numb']
pluto_cols_to_keep = ['bbl', 'block', 'borocode', 'condono']

In [115]:
finance_condos_only = pd.merge(finance, dtm[dtm_cols_to_keep],
    how='inner', left_on=['bbl'], right_on=['unit_bbl'])

In [116]:
finance_condos_only = pd.merge(pluto[pluto_cols_to_keep],
    finance_condos_only, how='inner',
    left_on=['borocode', 'block', 'condono'],
    right_on=['condo_boro', 'block', 'condo_numb'],
    suffixes=['_pluto', '_finance'])

In [117]:
finance_condos_only.columns

Index(['bbl_pluto', 'block', 'borocode', 'condono', 'sale_price', 'sale_date',
       'tax_class_at_time_of_sale', 'year_built', 'residential_units',
       'commercial_units', 'total_units', 'bbl_finance', 'unit_bbl',
       'condo_boro', 'condo_numb'],
      dtype='object')

In [21]:
finance_condos_only.shape[0] - finance_condos_only.drop_duplicates().shape[0]

0

In [118]:
finance_condos_only['bbl_pluto'] = finance_condos_only['bbl_pluto'].astype(int)

In [122]:
finance_condos_only = finance_condos_only[['bbl_pluto', 'bbl_finance']]

In [123]:
standard_bbls = list(set(finance.bbl).difference(
                        set(finance_condos_only.bbl_finance)))

In [128]:
bbl_mappings = finance_condos_only.append(pd.DataFrame.from_dict(
        {'bbl_pluto': standard_bbls, 'bbl_finance': standard_bbls}
    )).drop_duplicates()

In [129]:
bbl_mappings.shape[0] - bbl_mappings.drop_duplicates().shape[0]

0

In [132]:
finance_condo_updated = pd.merge(finance, bbl_mappings,
        how='left', left_on='bbl', right_on='bbl_finance')

In [134]:
finance_condo_updated.columns

Index(['sale_price', 'sale_date', 'tax_class_at_time_of_sale', 'year_built',
       'residential_units', 'commercial_units', 'total_units', 'block', 'bbl',
       'bbl_finance', 'bbl_pluto'],
      dtype='object')

In [135]:
pluto.columns

Index(['borough', 'block', 'schooldist', 'council', 'zipcode', 'ltdheight',
       'splitzone', 'bldgclass', 'landuse', 'easements', 'ownertype',
       'comarea', 'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'lotfront', 'lotdepth', 'bldgfront', 'bldgdepth', 'proxcode',
       'irrlotcode', 'lottype', 'bsmtcode', 'yearbuilt', 'builtcode',
       'histdist', 'landmark', 'borocode', 'bbl', 'condono', 'xcoord',
       'ycoord', 'zonemap', 'latitude', 'longitude', 'gross_sqft_pluto',
       'garage', 'extension', 'countalter'],
      dtype='object')